# State

Start by defining the State of the graph.

This state schema acts as the input structure for all Nodes and Edges within the graph.

In [ ]:
from typing import TypedDict

class State(TypedDict):
    graph_state: str

# Nodes

**Nodes are simply Python functions.**

Each node takes the state as its first positional argument, based on the previously defined `TypedDict` schema.

Since the state includes a `graph_state` key, each node can access it using `state['graph_state']`.

> Each node returns an updated value for graph_state, and by default, this new value will overwrite the existing one in the state.

In [ ]:
def node_1(state):
    print("---Node 1---")
    return {"graph_state": state['graph_state'] +" Welcome"}

def node_2(state):
    print("---Node 2---")
    return {"graph_state": state['graph_state'] +" to the DataHack Summit!"}

def node_3(state):
    print("---Node 3---")
    return {"graph_state": state['graph_state'] +" to Bengaluru!"}


# Edges

Edges define the connections between nodes in the graph.

* **Normal edges** are used when you always want to transition from one node to another—for example, from `node_1` to `node_2`.

* **Conditional edges** allow for **dynamic routing** based on logic. These are implemented as functions that evaluate the current state and return the name of the next node to execute.

In [ ]:
import random
from typing import Literal

def decide_node(state) -> Literal["node_2", "node_3"]:

    user_input = state['graph_state']

    if random.random() < 0.5:
        return "node_2"

    return "node_3"

# Graph Workflow


In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Build graph
builder = StateGraph(State)

# Defining the nodes
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Defining the edges
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_node)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Compile the Graph
graph = builder.compile()

# Display the Graph
display(Image(graph.get_graph().draw_mermaid_png()))

# Graph Execution


In [ ]:
graph.invoke({"graph_state" : "Hi there, it's Parthiban K!"})

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages: list[AnyMessage]

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(MessagesState)

builder.add_node("tool_calling_llm", tool_calling_llm)

builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Hey there! I'm Parthiban K")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 10 and 3")})
for m in messages['messages']:
    m.pretty_print()

In [ ]:

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(MessagesState)

builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply]))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,
)
builder.add_edge("tools", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="Hello, what is 2 multiplied by 2?")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()